# Week 2 - Challenge
## LangChain Practice

### Advanced Customer Service Agent with LangChain

> Deliverables

>> 1. Implementation using LangChain Expression Language (LCEL) to build the agent.
>> 2. Pydantic models for QueryAnalysis and ConversationSummary fully integrated into the chain.
>> 3. Usage examples for each suggested test query, demonstrating the end-to-end functionality.
>> 4. Results analysis. Include a screenshot or public link to a LangSmith trace for one of the complex queries to demonstrate successful tracing.

#### Setup & API Configuration

In [48]:
# Install packages

!pip install openai
!pip install python-dotenv
!pip install -qU \
  langchain-core \
  langchain-openai \
  langchain-community \
  langsmith

In [49]:
# Import libraries

import os
from getpass import getpass
import openai
from langchain_openai import ChatOpenAI
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from langchain.schema.runnable import RunnableLambda#, RunnablePassthrough
from pydantic import BaseModel, Field, ConfigDict
from typing import Literal, Optional, List
import json
import datetime
from uuid import uuid4 # Generate unique IDs for each conversation

In [50]:
# Load environment variables from .env file

# OPENAI_API_KEY

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
or getpass("Enter OpenAI API Key: ")

# LANGCHAIN_API_KEY

os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY") \
or getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "Advanced-Customer-Agent"

client = openai.OpenAI()

# Define the OpenAI model to use

openai_model = "gpt-4o-mini"

# Initialize the OpenAI chat model

llm = ChatOpenAI(model=openai_model,temperature=0)

In [51]:
test_queries = [
    "Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"
    ,"Urgent-Negative: This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!"
    ,"Satisfied-Positive: Thank you so much for the excellent service with my previous purchase. I want to buy gaming headphones"
    ,"Frustrated-Technical: I can't configure the router I bought last week, I've tried everything and it doesn't work"
    ,"Formal-Billing: Good morning, I need the receipt for my purchase from December 15th, order #TEC-2023-089"
    ,"Warranty-Query: I bought a tablet 8 months ago and now it won't turn on, how do I use the warranty?"
]

#### Pydantic Objects

In [52]:
# Define the data model for the extracted information

class ExtractedEntities(BaseModel):
    product_name: Optional[str] = Field(None, description="The specific product mentioned by the user")
    order_number: Optional[str] = Field(None, description="The order number mentioned by the user")
    date_info: Optional[str] = Field(None, description="The date mentioned by the user, use the days of the week, and not 'today,' 'tomorrow' or 'yesterday.' (only if applicable)")
    
    model_config = ConfigDict(extra="forbid")

In [53]:
# Define the query analysis model

class QueryAnalysis(BaseModel):
    """Analyzes and classifies a customer query."""
    query_category: Literal["technical_support","billing","returns","product_inquiry","general_information"]
    urgency_level: Literal["low","medium","high"]
    customer_sentiment: Literal["positive","neutral","negative"]
    entities: ExtractedEntities  

In [54]:
# Define the prompt template to analyze the user message and extract relevant information

class ConversationSummary(BaseModel):
    """A structured summary of the customer service interaction."""
    timestamp: str
    customer_id: str = "auto_generated"
    conversation_summary: str = Field(description="A concise, one-sentence summary of the interaction.")
    query_category: str
    customer_sentiment: str
    urgency_level: str
    mentioned_products: List[str]
    extracted_information: ExtractedEntities
    resolution_status: Literal["resolved", "pending", "escalated"]
    actions_taken: List[str] = Field(description="A list of actions the agent took or suggested.")
    follow_up_required: bool
    
    model_config = ConfigDict(extra="forbid")

#### 1° Component - Query Analysis & Classification

Analyze the initial User message and extract key information into a structured format.

In [55]:
# Model selected

print(llm.model_name)

gpt-4o-mini


In [56]:
# Sample of the first query

user_message = test_queries[0]
user_message

"Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago"

In [57]:
# Define the system prompt for the chatbot
# This prompt sets the context and behavior of the chatbot

system_prompt = SystemMessagePromptTemplate.from_template(
"""
You are an intelligent assistant that analyzes customer service messages and classifies them with the following criteria:
    1. Analyze the customer message and extract relevant information based on the pydantic model "QueryAnalysis".
    2. Be accurate in your classification.
    3. Respond ONLY with valid JSON structure
""")

print(system_prompt.format().content)


You are an intelligent assistant that analyzes customer service messages and classifies them with the following criteria:
    1. Analyze the customer message and extract relevant information based on the pydantic model "QueryAnalysis".
    2. Be accurate in your classification.
    3. Respond ONLY with valid JSON structure



In [58]:
# Create a specific prompt for the user message analysis

user_prompt = HumanMessagePromptTemplate.from_template(
"""
Analyze the customer message: {user_message}
""", 
    input_variables=["user_message"],
)

print(user_prompt.format(user_message=user_message).content)


Analyze the customer message: Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago



In [59]:
# Create the prompt template for analysis and classification

analysis_classification_prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt
])

# Create the chain
# This chain will perform the steps: prompt formatting > llm generation > get output in JSON format.

analysis_classification_chain = (
    analysis_classification_prompt
    | llm.with_structured_output(QueryAnalysis)
)

# Test the chain

analysis_classification_chain_test = analysis_classification_chain.invoke({"user_message": user_message})

print(analysis_classification_chain_test.model_dump_json(indent=2))

{
  "query_category": "product_inquiry",
  "urgency_level": "medium",
  "customer_sentiment": "neutral",
  "entities": {
    "product_name": "iPhone 15",
    "order_number": null,
    "date_info": null
  }
}


#### 2° Component - Dynamic Response Generation

Generate a context-aware, personalized response.

In [60]:
print(test_queries[0])

Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago


In [61]:
system_category_prompts = {
    "technical_support": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be empathetic, patient, and ask for relevant troubleshooting steps.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "billing": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be clear and polite, asking for details like invoice number or payment method.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "returns": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be understanding and explain the return process, asking for order number if needed.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "product_inquiry": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be helpful and informative, providing accurate product details.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "product_inquiry": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be helpful and informative, providing accurate product details.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "product_inquiry": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be helpful and informative, providing accurate product details.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """),

    "general_information": PromptTemplate.from_template(
        """
        You are a helpful and friendly assistant for TechStore Plus that should be courteous and provide concise, relevant information or guidance.
        User message: {user_message}
        Analysis: {analysis}
        Customer sentiment: {customer_sentiment}
        Extracted info: {entities}
        """)
}

In [62]:
# Iterate over the test queries and analyze each one

for query in test_queries:

    analysis_result = analysis_classification_chain.invoke({"user_message": query})

    # Dynamic responses

    prompt_template = system_category_prompts[analysis_result.query_category]
    formatted_prompt = prompt_template.format(
        user_message=query,
        analysis=analysis_result.query_category,
        customer_sentiment=analysis_result.customer_sentiment,
        entities=analysis_result.entities.model_dump()
    )
    assistant_response = llm.invoke(formatted_prompt)

    # Print the results

    print("\n--- QUERY ---")
    print(query)

    print("\n--- ASSISTANT RESPONSE ---")
    print(assistant_response.content)

    print("\n--- STRUCTURED OUTPUT ---")
    analysis_json = {
        "query_category": analysis_result.query_category,
        "urgency_level": analysis_result.urgency_level,
        "customer_sentiment": analysis_result.customer_sentiment,
        "entities": {
            "product_name": analysis_result.entities.product_name,
            "order_number": analysis_result.entities.order_number,
            "date_info": analysis_result.entities.date_info
        }
    }
    print(json.dumps(analysis_json, indent=2))


--- QUERY ---
Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago

--- ASSISTANT RESPONSE ---
Hello! Yes, we do have the new iPhone 15 in stock. As for shipping costs to Chicago, it typically depends on the shipping method you choose. Standard shipping usually costs around $5, while expedited options may vary. If you’d like, I can help you with the ordering process or provide more details about the iPhone 15!

--- STRUCTURED OUTPUT ---
{
  "query_category": "product_inquiry",
  "urgency_level": "medium",
  "customer_sentiment": "neutral",
  "entities": {
    "product_name": "iPhone 15",
    "order_number": null,
    "date_info": null
  }
}

--- QUERY ---
Urgent-Negative: This is an emergency! My order #TEC-2024-001 never arrived and I need that laptop for work tomorrow!

--- ASSISTANT RESPONSE ---
I'm really sorry to hear that your laptop hasn't arrived yet, especially with your work deadline approaching. I underst

#### 3° Component - Conversation Summarization & Persistance

Generate a structured summary of the entire interaction, ready for logging.

In [63]:
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an assistant that writes a structured one-sentence summary of a customer service interaction.
Return a JSON that strictly matches the ConversationSummary schema. 
- Use the timestamp and customer_id provided.
- Use analysis_json for query_category, urgency_level, sentiment, and entities.
- "mentioned_products" should list any product names found.
- "extracted_information" should include order numbers, dates, or other key details (dict).
- "resolution_status" must be one of: resolved, pending, escalated (choose realistically).
- "actions_taken" should be a short list of what the assistant actually did/suggested in the reply.
- "follow_up_required" is True if the user still needs to do something or if the case is unresolved."""),
    ("human", """User query:
{user_query}

Assistant reply:
{assistant_reply}

Analysis (JSON):
{analysis_json}

Timestamp: {timestamp}
Customer ID: {customer_id}""")
])

summary_chain = summary_prompt | llm.with_structured_output(ConversationSummary)

In [66]:
for query in test_queries:
        
    analysis_result = analysis_classification_chain.invoke({"user_message": query})

    prompt_template = system_category_prompts[analysis_result.query_category]
    formatted_prompt = prompt_template.format(
        user_message=query,
        analysis=analysis_result.query_category,
        customer_sentiment=analysis_result.customer_sentiment,
        entities=analysis_result.entities.model_dump()
    )
    assistant_response = llm.invoke(formatted_prompt)

    analysis_json = {
        "query_category": analysis_result.query_category,
        "urgency_level": analysis_result.urgency_level,
        "customer_sentiment": analysis_result.customer_sentiment,
        "entities": {
            "product_name": analysis_result.entities.product_name,
            "order_number": analysis_result.entities.order_number,
            "date_info": analysis_result.entities.date_info
        }
    }

    print("\n--- QUERY ---")
    print(query)
    print("\n--- ASSISTANT RESPONSE ---")
    print(assistant_response.content)
    print("\n--- STRUCTURED OUTPUT ---")
    print(json.dumps(analysis_json, indent=2))

    timestamp_str = datetime.datetime.now().isoformat()
    conversation_summary = summary_chain.invoke({
        "user_query": query,
        "assistant_reply": getattr(assistant_response, "content", str(assistant_response)),
        "analysis_json": json.dumps(analysis_json, ensure_ascii=False, indent=2),
        "timestamp": timestamp_str,
        "customer_id": str(uuid4())
    })

    print("\n=== CONVERSATION SUMMARY ===")
    print(conversation_summary.model_dump_json(indent=2))


--- QUERY ---
Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago

--- ASSISTANT RESPONSE ---
Hello! Yes, we do have the new iPhone 15 in stock. As for shipping costs to Chicago, it typically depends on the shipping method you choose. Standard shipping usually costs around $5, while expedited options may vary. If you’d like more specific details or to place an order, feel free to let me know!

--- STRUCTURED OUTPUT ---
{
  "query_category": "product_inquiry",
  "urgency_level": "medium",
  "customer_sentiment": "neutral",
  "entities": {
    "product_name": "iPhone 15",
    "order_number": null,
    "date_info": null
  }
}

=== CONVERSATION SUMMARY ===
{
  "timestamp": "2025-08-09T01:18:06.118682",
  "customer_id": "601fb30c-ca42-4f31-8733-2a20d35b222b",
  "conversation_summary": "The customer inquired about the availability and shipping costs of the new iPhone 15 to Chicago, and the assistant confirmed it is in st

#### 4° LangSmith Integration

Observability and monitoring

In [65]:
print("LangSmith -- Tracing Projects -- Advanced-Customer-Agent: \n\n" \
">> Test chain on customer message: \n" \
"Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago \n\n"
"https://smith.langchain.com/o/ca30c0fa-d39b-4e14-892a-aa6b8b3f7caa/projects/p/7b350e83-86d5-43b2-b195-a83cb9d98184?timeModel=%7B%22duration%22%3A%227d%22%7D")

LangSmith -- Tracing Projects -- Advanced-Customer-Agent: 

>> Test chain on customer message: 
Neutral-Informative: Hello, I'd like to know if you have the new iPhone 15 in stock and how much shipping costs to Chicago 

https://smith.langchain.com/o/ca30c0fa-d39b-4e14-892a-aa6b8b3f7caa/projects/p/7b350e83-86d5-43b2-b195-a83cb9d98184?timeModel=%7B%22duration%22%3A%227d%22%7D
